In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from pprint import pprint

In [ ]:
from nltk import word_tokenize

<br>Lemmatize
<br>Stopwords removal

In [ ]:

import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import  wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

#RemoveStopWordsFunction
def RemoveStopWords(string):
    #Removing Punctuations
    for each in punctuation:
        string=string.replace(each,"")
    
    #Removing Stopwords
    english_stopwords=stopwords.words('english')
    stopwords_removed_tokens=[]
    words=string.split(" ")
    
    for each in words:
        if each not in english_stopwords:
            stopwords_removed_tokens.append(each)
    return " ".join(stopwords_removed_tokens) 


#LemmatizeFunction
def Lemmatize(string):
    word_lem=WordNetLemmatizer()
    words=string.split() 
    lemmatizeWords=[]
    for each in words:
        try:
            lemmatizeWords.append(word_lem.lemmatize(each))
        except:
            continue
    return " ".join(lemmatizeWords)

def Refine(string):
    return Lemmatize(RemoveStopWords(string))
    #return Lemmatize(RemoveStopWords(Tokenize(string)))


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sgudipudi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Load the data from all sub folders into CSV

In [ ]:
#Read all the files from all subfolders and store their summaries in a single CSV file.
#Name CSV File as: “BBCNewsArticles.csv”

import os
data=[]
for path,folders,files in os.walk('BBC News Articles'):
    for each in files:
        if each.endswith('txt'):
            f=open(path+"//"+each,'r')
            content=f.read()
            data.append(content)
            f.close()
        
import pandas as pd
df=pd.DataFrame(data,columns=['Article'])
df.to_csv('BBCNewsArticles.csv',index=False)    

Shuffle the rows

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

Apply the refine function(which does stop words removing, lemmatization etc) on every row with help of lambda function

In [ ]:
df['Refined_Article']=list(map(lambda x: Refine(x),df['Article']))

In [ ]:
df.head()

,Article,Refined_Article
118,Brown ally rejects Budget spree\n\nChancellor ...,Brown ally reject Budget spree Chancellor Gord...
134,UK 'needs true immigration data'\n\nA former H...,UK need true immigration data A former Home Of...
94,Off-colour Gardener storms to win\n\nBritain's...,Offcolour Gardener storm win Britains Jason Ga...
37,Redford's vision of Sundance\n\nDespite sporti...,Redfords vision Sundance Despite sporting cord...
15,Bennett play takes theatre prizes\n\nThe Histo...,Bennett play take theatre prize The History Bo...


In [ ]:
len(df)

179

Lower Case the refined articles

In [ ]:
reviews_lower = [sent.lower() for sent in df.Refined_Article.values]
reviews_lower[0]

'brown ally reject budget spree chancellor gordon browns closest ally denied suggestion budget giveaway 16 march ed balls exchief economic adviser treasury said would spending spree polling day but mr balls prospective labour mp said confident chancellor would meet fiscal rule he speaking sir digby jones cbi director general warned mr brown tempted use extra cash preelection bribe mr balls stepped treasury post stand labour candidate election suggested mr brown would meet golden economic rule margin spare he said hoped would done build current tax credit rule he also stressed rise interest rate ahead expected may election would affect labour partys chance winning expectations rate rise gathered pace figure showed house price still rising consumer borrowing rose nearrecord pace january if mpc bank englands monetary policy committee judge rate rise justified election strength economy im predicting i believe big election issue britain labour told parliamentary lunch this big change politi

In [ ]:
reviews_lower[1]

'uk need true immigration data a former home office minister called independent body set monitor uk immigration barbara roche said organisation monitor publish figure independent government she said would counter socalled independent group like migration watch described antiimmigration body posing independent migration watch say immigration government already publishes accurate figure sir andrew green chairman organisation say need independent body office national statistics data accurate he say opposes largescale immigration ground overcrowding culture he said for example next 20 year one household three due immigration we already overcrowded india four time overcrowded france ms roche labour mp hornsey wood green belief legal migration something welcome she said proposal mean wouldnt socalled independent expert like migration watch come debate antiimmigration point view she went what i would like see body actually looked figure published independent government i think would go long w

In [ ]:
news_token = [word_tokenize(sent) for sent in reviews_lower]
news_token[1]

['uk',
 'need',
 'true',
 'immigration',
 'data',
 'a',
 'former',
 'home',
 'office',
 'minister',
 'called',
 'independent',
 'body',
 'set',
 'monitor',
 'uk',
 'immigration',
 'barbara',
 'roche',
 'said',
 'organisation',
 'monitor',
 'publish',
 'figure',
 'independent',
 'government',
 'she',
 'said',
 'would',
 'counter',
 'socalled',
 'independent',
 'group',
 'like',
 'migration',
 'watch',
 'described',
 'antiimmigration',
 'body',
 'posing',
 'independent',
 'migration',
 'watch',
 'say',
 'immigration',
 'government',
 'already',
 'publishes',
 'accurate',
 'figure',
 'sir',
 'andrew',
 'green',
 'chairman',
 'organisation',
 'say',
 'need',
 'independent',
 'body',
 'office',
 'national',
 'statistics',
 'data',
 'accurate',
 'he',
 'say',
 'opposes',
 'largescale',
 'immigration',
 'ground',
 'overcrowding',
 'culture',
 'he',
 'said',
 'for',
 'example',
 'next',
 '20',
 'year',
 'one',
 'household',
 'three',
 'due',
 'immigration',
 'we',
 'already',
 'overcrowded',
 

In [ ]:
id2word = corpora.Dictionary(news_token)
texts = news_token
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 6), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 3), (28, 1), (29, 5), (30, 2), (31, 3), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 5), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 3), (57, 1), (58, 1), (59, 5), (60, 1), (61, 2), (62, 2), (63, 1), (64, 6), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 3), (86, 2), (87, 1), (88, 1), (89, 5), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 2), (101, 1), (102, 1), (103, 1), (104, 5), (105, 1), (106, 1), (107, 1), (108, 2), (109, 2), (110, 1)

In [ ]:
id2word[4]

'add'

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('16', 1),
  ('2001', 1),
  ('4', 1),
  ('5', 1),
  ('add', 1),
  ('added', 1),
  ('adviser', 1),
  ('affect', 1),
  ('ahead', 1),
  ('ally', 2),
  ('also', 1),
  ('amid', 1),
  ('anything', 1),
  ('anywhere', 1),
  ('assurance', 1),
  ('away', 2),
  ('backwards', 1),
  ('balls', 6),
  ('bank', 1),
  ('bbc', 1),
  ('believe', 1),
  ('betting', 1),
  ('big', 2),
  ('blair', 1),
  ('boring', 1),
  ('borrowing', 1),
  ('bribe', 1),
  ('britain', 3),
  ('british', 1),
  ('brown', 5),
  ('browns', 2),
  ('budget', 3),
  ('build', 1),
  ('business', 1),
  ('but', 2),
  ('came', 1),
  ('candidate', 1),
  ('cash', 2),
  ('cbi', 1),
  ('certainly', 1),
  ('chance', 1),
  ('chancellor', 5),
  ('change', 1),
  ('closest', 1),
  ('committee', 1),
  ('competitiveness', 1),
  ('confident', 1),
  ('conservative', 1),
  ('consumer', 1),
  ('credit', 1),
  ('culture', 1),
  ('current', 1),
  ('date', 1),
  ('day', 1),
  ('delivers', 1),
  ('denied', 1),
  ('digby', 3),
  ('director', 1),
  ('done', 1

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=42,
                                           passes=100,
                                           per_word_topics=True)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.016*"the" + 0.011*"said" + 0.006*"people" + 0.005*"it" + 0.005*"mobile" + '
  '0.005*"year" + 0.004*"one" + 0.004*"use" + 0.004*"i" + 0.004*"network"'),
 (1,
  '0.012*"said" + 0.011*"the" + 0.008*"mr" + 0.006*"would" + 0.004*"us" + '
  '0.004*"also" + 0.003*"new" + 0.003*"year" + 0.003*"job" + 0.003*"bbc"'),
 (2,
  '0.012*"said" + 0.012*"the" + 0.008*"world" + 0.006*"i" + 0.005*"year" + '
  '0.005*"mr" + 0.005*"government" + 0.003*"would" + 0.003*"olympic" + '
  '0.003*"country"'),
 (3,
  '0.014*"the" + 0.012*"said" + 0.006*"email" + 0.006*"i" + 0.005*"mr" + '
  '0.005*"year" + 0.004*"new" + 0.003*"microsoft" + 0.003*"last" + '
  '0.003*"program"'),
 (4,
  '0.021*"the" + 0.011*"said" + 0.006*"year" + 0.005*"best" + 0.005*"also" + '
  '0.004*"film" + 0.004*"new" + 0.004*"i" + 0.004*"would" + 0.003*"world"')]


In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=news_token, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.2431754623226116


In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.30984448466318


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/envs/training/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.103739  0.019246       1        1  29.912716
4     -0.105176  0.070031       2        1  21.799170
1      0.035689  0.065868       3        1  17.918472
3      0.022684 -0.073035       4        1  17.146202
2     -0.056936 -0.082109       5        1  13.223436, topic_info=     Category        Freq        Term       Total  loglift  logprob
2644  Default   47.000000       email   47.000000  30.0000  30.0000
1563  Default   58.000000      mobile   58.000000  29.0000  29.0000
1053  Default   44.000000     network   44.000000  28.0000  28.0000
448   Default  121.000000       world  121.000000  27.0000  27.0000
1583  Default   40.000000       phone   40.000000  26.0000  26.0000
312   Default   68.000000        best   68.000000  25.0000  25.0000
3023  Default   34.000000     digital   34.000000  24.0000  24.0000
2649  Default   34.000000   microsoft   34.000000  23.0000  23.0000
1150  Default   31.000000   athletics   31.000000  22.0000  22.0000
2656  Default   32.000000     program   32.000000  21.0000  21.0000
396   Default   32.000000     olympic   32.000000  20.0000  20.0000
115   Default  162.000000          mr  162.000000  19.0000  19.0000
307   Default   31.000000      athens   31.000000  18.0000  18.0000
1132  Default   44.000000        user   44.000000  17.0000  17.0000
365   Default   26.000000      indoor   26.000000  16.0000  16.0000
23    Default   35.000000       blair   35.000000  15.0000  15.0000
6547  Default   24.000000      ballet   24.000000  14.0000  14.0000
321   Default   30.000000    champion   30.000000  13.0000  13.0000
2932  Default   21.000000       virus   21.000000  12.0000  12.0000
5750  Default   24.000000         ink   24.000000  11.0000  11.0000
84    Default   85.000000  government   85.000000  10.0000  10.0000
410   Default   40.000000      record   40.000000   9.0000   9.0000
3869  Default   23.000000      gadget   23.000000   8.0000   8.0000
1950  Default   19.000000        drug   19.000000   7.0000   7.0000
99    Default   37.000000         job   37.000000   6.0000   6.0000
89    Default  157.000000           i  157.000000   5.0000   5.0000
3110  Default   18.000000     blogger   18.000000   4.0000   4.0000
153   Default  453.000000        said  453.000000   3.0000   3.0000
1958  Default   16.000000       greek   16.000000   2.0000   2.0000
408   Default   32.000000        race   32.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
997    Topic5   10.700912   education   20.897917   1.3539  -6.1694
5949   Topic5    5.258476       5000m    7.868455   1.6202  -6.8799
365    Topic5   12.289220      indoor   26.512463   1.2543  -6.0310
410    Topic5   16.338581      record   40.250469   1.1216  -5.7462
389    Topic5    6.192187   medallist   10.143860   1.5296  -6.7164
23     Topic5   14.027781       blair   35.494759   1.0948  -5.8987
84     Topic5   24.239067  government   85.672195   0.7606  -5.3518
153    Topic5   63.422749        said  453.329498   0.0564  -4.3899
1500   Topic5    7.660171      africa   14.198004   1.4061  -6.5037
432    Topic5   63.382500         the  591.641602  -0.2105  -4.3905
1037   Topic5    9.969286       light   23.165417   1.1800  -6.2402
89     Topic5   28.513287           i  157.595825   0.3135  -5.1893
1201   Topic5   16.201059      report   57.215408   0.7614  -5.7546
264    Topic5   12.002550         she   33.533623   0.9957  -6.0546
1165   Topic5   17.225847     country   69.732712   0.6249  -5.6933
111    Topic5   12.407309    minister   37.771217   0.9099  -6.0214
1713   Topic5    9.742662      family   23.310024   1.1508  -6.2632
115    Topic5   24.735483          mr  162.180176   0.1427  -5.3315
280    Topic5   25.198139        year  184.993271   0.0296  -5.3129
134    Topic5    7.310287       prime   14.026583   1.3715  -6.5504
85     Topic5   15.797066          he   81.

<p>How to interpret coherence scores based on c_v scoring</p>
<br>.3 is bad

<br>.4 is low

<br>.55 is okay

<br>.65 might be as good as it is going to get

<br>.7 is nice

<br>.8 is unlikely and

<br>.9 is probably wrong